In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import snowflake.connector
import warnings
from redmail import gmail
from simple_colors import *
import datetime as dt
import requests
import json
import math
import seaborn as sns
from IPython.display import display, Javascript
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
os.chdir('/Users/jessherbert/Desktop/Critical/Dashboards')
# Connect to Snowflake Databases
conn_recurly=snowflake.connector.connect(user='JESSHERBERT',password='SpartanLife308',account='soa50355',warehouse='COMPUTE_WH',database='DAILYWIRE_RECURLY',schema='CLASSIC')

In [2]:
data = conn_recurly.cursor().execute("""

WITH SUBS AS (SELECT U.SUBSCRIPTION_UUID SUB_ID, U.ACCOUNT_CODE USER_ID, S.STATE, U.ORIGINAL_PLAN_CODE OG,
                     U.CURRENT_PLAN_CODE CRNT, U.RECENT_PLAN_CODE RECENT, S.EMAIL,
                     DATEADD(HOURS,-5,S.ACTIVATED_AT) ACT, DATEADD(HOURS,-5,S.CANCELED_AT) CNCL,
                     DATEADD(HOURS,-5,S.EXPIRES_AT) EXP,
                     (CASE WHEN EXP IS NULL THEN DATEDIFF(MONTHS,ACT,CURRENT_DATE())
                           ELSE DATEDIFF(MONTHS,ACT,EXP)
                      END) L1, 
                     (CASE WHEN EXP IS NULL THEN (CASE WHEN DAY(CURRENT_DATE()) >= DAY(ACT) THEN 0
                                                       ELSE -1
                                                  END)
                           ELSE (CASE WHEN DAY(EXP) >= DAY(ACT) THEN 0
                                      ELSE -1
                                 END)
                      END) ADJ,
                      L1 + ADJ LENGTH, DATEDIFF(MONTHS,ACT, CURRENT_DATE()) PL1, PL1 + ADJ POSSIBLE_LENGTH,
                      U.COUPON_CODE, ZEROIFNULL(U.DISCOUNT_PERCENT) COUPON_PCT, ZEROIFNULL(U.NET_REVENUE) NET_REV,
                      ZEROIFNULL(U.MONTHLY_RENEWALS) MONTHLY_RENEWALS,
                      ZEROIFNULL(U.ANNUAL_RENEWALS) ANNUAL_RENEWALS
              FROM DMITRY.DMITRY_SB.USER_PROFILE U
              LEFT JOIN DAILYWIRE_RECURLY.CLASSIC.SUBSCRIPTIONS S
              ON U.SUBSCRIPTION_UUID = S.UUID AND U.ACCOUNT_CODE = S.ACCOUNT_CODE
              ORDER BY ACT
              )
SELECT SUB_ID, OG, DATE_FROM_PARTS(YEAR(ACT),MONTH(ACT),1) COHORT, LENGTH, POSSIBLE_LENGTH, NET_REV FROM SUBS;

""").fetch_pandas_all()

In [3]:
startDate=dt.date(2020,1,1)

ALL = data[data.COHORT >= startDate]
RP = data[(data.OG == 'reader') & (data.COHORT >= startDate)]
IM = data[(data.OG == 'insider') & (data.COHORT >= startDate)]
IA = data[(data.OG == 'insider_plus') & (data.COHORT >= startDate)]
AA = data[(data.OG == 'all_access') & (data.COHORT >= startDate)]

In [4]:
### ALL PLANS RETENTION MATRIX
ALLretentionMatrix = pd.DataFrame([])

for i in sorted(set(ALL.COHORT.values)):
    # LIMIT DATA TO MONTH X COHORT
    cohort = ALL[ALL.COHORT == i]
    
    # COUNT SUBSCRIPTIONS STARTED DURING MONTH X
    starting = len(cohort)
    
    # BUILD DICTIONARY AND DATAFRAME
    DIC = {'COHORT':i}
    keys = range(-1, cohort.POSSIBLE_LENGTH.values.max())
    rowData = []
    
    # FOR EACH POSSIBLE LENGTH, COUNT REMAINING SUBSCRIPTIONS
    for l in range(-1, cohort.POSSIBLE_LENGTH.values.max()):
        users = len(cohort[cohort.LENGTH > l])
        rowData = rowData + [users/starting]
        DIC[str((l+1))] = rowData[(l+1)]
    
    # USE DICTIONARY TO BUILD RETENTION ROW FOR MONTH X COHORT
    cohortRetention = pd.DataFrame(data = DIC, index = range(len(DIC)))[:1]
    
    # ADD COHORT TO RETENTION MATRIX
    ALLretentionMatrix = ALLretentionMatrix.append(cohortRetention)
    
ALLretentionMatrix = ALLretentionMatrix.drop(labels = '0', axis = 1)

strDates=[]
for i in range(len(ALLretentionMatrix)):
    dtDate = ALLretentionMatrix.COHORT.values[i]
    strDates.append(str(int(dtDate.month))+'/'+str(int(dtDate.year))[2:4])
ALLretentionMatrix['Subscription Activated'] = strDates

######################################################################################


### READERS PASS RETENTION MATRIX
RPretentionMatrix = pd.DataFrame([])

for i in sorted(set(RP.COHORT.values)):
    # LIMIT DATA TO MONTH X COHORT
    cohort = RP[RP.COHORT == i]
    
    # COUNT SUBSCRIPTIONS STARTED DURING MONTH X
    starting = len(cohort)
    
    # BUILD DICTIONARY AND DATAFRAME
    DIC = {'COHORT':i}
    keys = range(-1, cohort.POSSIBLE_LENGTH.values.max())
    rowData = []
    
    # FOR EACH POSSIBLE LENGTH, COUNT REMAINING SUBSCRIPTIONS
    for l in range(-1, cohort.POSSIBLE_LENGTH.values.max()):
        users = len(cohort[cohort.LENGTH > l])
        rowData = rowData + [users/starting]
        DIC[str((l+1))] = rowData[(l+1)]
    
    # USE DICTIONARY TO BUILD RETENTION ROW FOR MONTH X COHORT
    cohortRetention = pd.DataFrame(data = DIC, index = range(len(DIC)))[:1]
    
    # ADD COHORT TO RETENTION MATRIX
    RPretentionMatrix = RPretentionMatrix.append(cohortRetention)
    
RPretentionMatrix = RPretentionMatrix.drop(labels = '0', axis = 1)

strDates=[]
for i in range(len(RPretentionMatrix)):
    dtDate = RPretentionMatrix.COHORT.values[i]
    strDates.append(str(int(dtDate.month))+'/'+str(int(dtDate.year))[2:4])
RPretentionMatrix['Subscription Activated'] = strDates

######################################################################################

### INSIDER MONTHLY RETENTION MATRIX
IMretentionMatrix = pd.DataFrame([])

for i in sorted(set(IM.COHORT.values)):
    # LIMIT DATA TO MONTH X COHORT
    cohort = IM[IM.COHORT == i]
    #print(cohort)
    
    # COUNT SUBSCRIPTIONS STARTED DURING MONTH X
    starting = len(cohort)
    
    # BUILD DICTIONARY AND DATAFRAME
    DIC = {'COHORT':i}
    keys = range(-1, cohort.POSSIBLE_LENGTH.values.max())
    rowData = []
    
    # FOR EACH POSSIBLE LENGTH, COUNT REMAINING SUBSCRIPTIONS
    for l in range(-1, cohort.POSSIBLE_LENGTH.values.max()):
        users = len(cohort[cohort.LENGTH > l])
        rowData = rowData + [users/starting]
        DIC[str((l+1))] = rowData[(l+1)]
        #print(users)
    
    # USE DICTIONARY TO BUILD RETENTION ROW FOR MONTH X COHORT
    cohortRetention = pd.DataFrame(data = DIC, index = range(len(DIC)))[:1]
    
    # ADD COHORT TO RETENTION MATRIX
    IMretentionMatrix = IMretentionMatrix.append(cohortRetention)
    
IMretentionMatrix = IMretentionMatrix.drop(labels = '0', axis = 1)  

strDates=[]
for i in range(len(IMretentionMatrix)):
    dtDate = IMretentionMatrix.COHORT.values[i]
    strDates.append(str(int(dtDate.month))+'/'+str(int(dtDate.year))[2:4])
IMretentionMatrix['Subscription Activated'] = strDates

######################################################################################

### INSIDER ANNUAL RETENTION MATRIX
IAretentionMatrix = pd.DataFrame([])

for i in sorted(set(IA.COHORT.values)):
    # LIMIT DATA TO MONTH X COHORT
    cohort = IA[IA.COHORT == i]
    
    # COUNT SUBSCRIPTIONS STARTED DURING MONTH X
    starting = len(cohort)
    
    # BUILD DICTIONARY AND DATAFRAME
    DIC = {'COHORT':i}
    keys = range(-1, cohort.POSSIBLE_LENGTH.values.max())
    rowData = []
    
    # FOR EACH POSSIBLE LENGTH, COUNT REMAINING SUBSCRIPTIONS
    for l in range(-1, cohort.POSSIBLE_LENGTH.values.max()):
        users = len(cohort[cohort.LENGTH > l])
        rowData = rowData + [users/starting]
        DIC[str((l+1))] = rowData[(l+1)]
    
    # USE DICTIONARY TO BUILD RETENTION ROW FOR MONTH X COHORT
    cohortRetention = pd.DataFrame(data = DIC, index = range(len(DIC)))[:1]
    
    # ADD COHORT TO RETENTION MATRIX
    IAretentionMatrix = IAretentionMatrix.append(cohortRetention)
    
IAretentionMatrix = IAretentionMatrix.drop(labels = '0', axis = 1)  

strDates=[]
for i in range(len(IAretentionMatrix)):
    dtDate = IAretentionMatrix.COHORT.values[i]
    strDates.append(str(int(dtDate.month))+'/'+str(int(dtDate.year))[2:4])
IAretentionMatrix['Subscription Activated'] = strDates


######################################################################################


AAretentionMatrix = pd.DataFrame([])

for i in sorted(set(AA.COHORT.values)):
    # LIMIT DATA TO MONTH X COHORT
    cohort = AA[AA.COHORT == i]
    
    # COUNT SUBSCRIPTIONS STARTED DURING MONTH X
    starting = len(cohort)
    
    # BUILD DICTIONARY AND DATAFRAME
    DIC = {'COHORT':i}
    keys = range(-1, cohort.POSSIBLE_LENGTH.values.max())
    rowData = []
    
    # FOR EACH POSSIBLE LENGTH, COUNT REMAINING SUBSCRIPTIONS
    for l in range(-1, cohort.POSSIBLE_LENGTH.values.max()):
        users = len(cohort[cohort.LENGTH > l])
        rowData = rowData + [users/starting]
        DIC[str((l+1))] = rowData[(l+1)]
    
    # USE DICTIONARY TO BUILD RETENTION ROW FOR MONTH X COHORT
    cohortRetention = pd.DataFrame(data = DIC, index = range(len(DIC)))[:1]
    
    # ADD COHORT TO RETENTION MATRIX
    AAretentionMatrix = AAretentionMatrix.append(cohortRetention)
    
AAretentionMatrix = AAretentionMatrix.drop(labels = '0', axis = 1)  

strDates=[]
for i in range(len(AAretentionMatrix)):
    dtDate = AAretentionMatrix.COHORT.values[i]
    strDates.append(str(int(dtDate.month))+'/'+str(int(dtDate.year))[2:4])
AAretentionMatrix['Subscription Activated'] = strDates

In [5]:
### ALL PLAN RETENTION
#plt.figure(figsize = (20,10))
#plt.title('Subscriber Retention - All Plans', fontsize = 14, fontweight = 'bold')
#sns.heatmap(ALLretentionMatrix.round(decimals = 2).drop(labels = 'COHORT',
#            axis = 1).set_index('Subscription Activated')[:-1], annot = True,
#            vmin = 0, vmax = 1, cmap = 'RdYlGn', fmt = 'g')
#plt.tick_params(axis = 'both', which = 'major', labelbottom = True, labeltop = True,
#                bottom = True, top = True)

### READERS PASS RETENTION
#plt.figure(figsize = (20,10))
#plt.title('Readers Pass Subscriber Retention', fontsize = 14, fontweight = 'bold')
#sns.heatmap(RPretentionMatrix.round(decimals = 2).drop(labels = 'COHORT',
#            axis = 1).set_index('Subscription Activated')[:-1], annot = True,
#            vmin = 0, vmax = 1, cmap = 'RdYlGn', fmt = 'g')
#plt.tick_params(axis = 'both', which = 'major', labelbottom = True, labeltop = True,
#                bottom = True, top = True)


### INSIDER MONTHLY RETENTION
#plt.figure(figsize = (20,10))
#plt.title('Insider Monthly Subscriber Retention', fontsize = 14, fontweight = 'bold')
#sns.heatmap(IMretentionMatrix.round(decimals = 2).drop(labels = 'COHORT',
#            axis = 1).set_index('Subscription Activated')[:-1], annot = True,
#            vmin = 0, vmax = 1, cmap = 'RdYlGn', fmt = 'g')
#plt.tick_params(axis = 'both', which = 'major', labelbottom = True, labeltop = True,
#                bottom = True, top = True)

### INSIDER ANNUAL RETENTION
#plt.figure(figsize = (20,10))
#plt.title('Insider Annual Subscriber Retention', fontsize = 14, fontweight = 'bold')
#sns.heatmap(IAretentionMatrix.round(decimals = 2).drop(labels = 'COHORT',
#            axis = 1).set_index('Subscription Activated')[:-1], annot = True,
#            vmin = 0, vmax = 1, cmap = 'RdYlGn', fmt = 'g')
#plt.tick_params(axis = 'both', which = 'major', labelbottom = True, labeltop = True,
#                bottom = True, top = True)

### ALL ACCESS RETENTION
#plt.figure(figsize = (20,10))
#plt.title('All Access Subscriber Retention', fontsize = 14, fontweight = 'bold')
#sns.heatmap(AAretentionMatrix.round(decimals = 2).drop(labels = 'COHORT',
#            axis = 1).set_index('Subscription Activated')[:-1], annot = True,
#            vmin = 0, vmax = 1, cmap = 'RdYlGn', fmt = 'g')
#plt.tick_params(axis = 'both', which = 'major', labelbottom = True, labeltop = True,
#                bottom = True, top = True)

In [6]:
output = widgets.Output()
plotOutput = widgets.Output()

# SELECT PLAN
planSelection = widgets.ToggleButtons(
    options=['All','Readers Pass', 'Insider Monthly', 'Insider Annual','All Access'],
    description='Plan:',disabled=False,button_style='')

# SELECT START & END DATES
dateSelection1 = widgets.DatePicker(description = 'From:', disabled = False)
dateSelection2 = widgets.DatePicker(description = 'To:', disabled = False)


def filtering(plan, start, end):
    
    output.clear_output()
    plotOutput.clear_output()
    
    # WHEN START & END DATES HAVE BEEN SELECTED
    if (start is not None and end is not None):
        
        # BOTH DATES MUST BE 2015 or LATER
        if (start.year >= 2015 and end.year >= 2015):
            date1 = dt.date(start.year, start.month, 1)
            
            # END DATE MUSTN'T BE GREATER THAN CURRENT MONTH 
            if (end >= dt.date.today() or end < start):
                date2 = dt.date(dt.date.today().year,dt.date.today().month,1)
                
            else:
                date2 = dt.date(end.year, end.month, 1)
            
            # DEFINE MONTH LENGTHS TO SHOW IN HEATMAP
            possibleLength = ALL[ALL.COHORT >= date1].POSSIBLE_LENGTH.max()
            columnsList=['COHORT','Subscription Activated']
            for i in range(1,possibleLength+1):
                columnsList.append(str(i))
            
            # PLAN SELECTION AND PLOTTING
            if plan == 'All':
                data = ALLretentionMatrix[(ALLretentionMatrix.COHORT >= date1)&
                                         (ALLretentionMatrix.COHORT <= 
                                          date2)][columnsList] 
                c = 'k'
            
            if plan == 'Readers Pass':
                data = RPretentionMatrix[(RPretentionMatrix.COHORT >= date1)&
                                         (RPretentionMatrix.COHORT <= 
                                         date2)][columnsList]
                c = '#2676a5'
                    
            if plan == 'Insider Monthly':
                data = IMretentionMatrix[(IMretentionMatrix.COHORT >= date1)&
                                         (IMretentionMatrix.COHORT <= 
                                         date2)][columnsList]
                c = '#f1dc5b'
                    
            if plan == 'Insider Annual':
                data = IAretentionMatrix[(IAretentionMatrix.COHORT >= date1)&
                                         (IAretentionMatrix.COHORT <= 
                                         date2)][columnsList]
                c = '#723180'

            if plan == 'All Access':
                data = AAretentionMatrix[(AAretentionMatrix.COHORT >= date1)&
                                         (AAretentionMatrix.COHORT <= 
                                         date2)][columnsList]
                c = '#2da07b'
                
            data2 = data[:1].transpose().rename(columns={0:'PCT'})
            data2Labels=[]
            for i in range(len(data2[2:])):
                pct = data2[2:].PCT.values[i]
                if pct >= .995:
                    data2Labels.append(str(100*round(pct,2))[:3]+'%')
                elif pct >= .1:
                    data2Labels.append(str(100*round(pct,2))[:2]+'%')
                else:
                    data2Labels.append(str(100*round(pct,2))[:1]+'%')
            
            
            with output:
                display(data)
                warnings.filterwarnings("ignore")
                
            with plotOutput:
                
                # STARTING MONTH COHORT RETENTION LINE
                plt.figure (figsize = (20,5))
                plt.plot(range(len(data2[2:])), data2[2:].PCT.values,
                        color = c)
                for i in range(len(data2[2:])):
                    plt.annotate(data2Labels[i],
                                (i-.025,data2[2:].PCT.values[i]*1.0125))
                
                plt.title('Subscriber Retention - '+str(plan)+'\n(Activated '+
                          str(date1.month)+'/'+str(date1.year)[-2:]+')',
                         fontsize = 14, fontweight = 'bold')
                plt.ylim(bottom = 0, top = 1.05)
                plt.yticks(list(np.arange(0,1.1,step=.1)),
                          labels= ['0%','','20%','','40%','','60%','','80%','','100%'])
                plt.ylabel('Cohort Remaining (%)')
                plt.xticks(np.arange(len(data2[2:])),
                          np.arange(1, len(data2[2:])+1))
                plt.xlabel('Months Retained')
                plt.show()
                
                ### RETENTION MATRIX
                plt.figure (figsize = (20,10))
                plt.title('Subscriber Retention - '+str(plan), fontsize = 14,
                          fontweight = 'bold')
                sns.heatmap(data[data.COHORT!=dt.date(dt.date.today().year,
                            dt.date.today().month,
                            1)].drop(labels = 'COHORT',
                            axis = 1).round(decimals = 2).set_index('Subscription Activated'),
                               annot = True, vmin = 0, vmax = 1, cmap = 'RdYlGn',fmt = 'g',
                           linecolor='k', linewidths= .5)
                plt.tick_params(axis = 'both', which = 'major',
                                   labelbottom = True, bottom = True,
                                   labeltop = True, top = True)
                plt.xlabel('Months Retained')
                plt.show()
                
            
        
### EVENT HANDLERS
def planHandler(change):
    filtering(change.new, dateSelection1.value, dateSelection2.value)
    
def date1Handler(change):
    filtering(planSelection.value, change.new, dateSelection2.value)
    
def date2Handler(change):
    filtering(planSelection.value, dateSelection1.value, change.new)
    
### OBSERVE OUTPUT
planSelection.observe(planHandler, names = 'value')
dateSelection1.observe(date1Handler, names = 'value')
dateSelection2.observe(date2Handler, names = 'value')

In [7]:
display(planSelection, dateSelection1, dateSelection2, plotOutput)

ToggleButtons(description='Plan:', options=('All', 'Readers Pass', 'Insider Monthly', 'Insider Annual', 'All A…

DatePicker(value=None, description='From:')

DatePicker(value=None, description='To:')

Output()

In [8]:
display(output)

Output()